In [1]:
TRAIN_PATH = "Train"
VAL_PATH = "Val"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import * 
from keras.preprocessing import image

In [3]:
# CNN Based Model in Keras

model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 110, 110, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                    

In [5]:
# Train from scratch
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_dataset = image.ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')


Found 224 images belonging to 2 classes.


In [7]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [8]:
validation_generator = test_dataset.flow_from_directory(
    VAL_PATH,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')

Found 60 images belonging to 2 classes.


In [10]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=6,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps=2
)

C:\Users\hp\AppData\Local\Temp/ipykernel_13676/4052043046.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(


Epoch 1/10
6/6 [==============================] - 49s 8s/step - loss: 0.6522 - accuracy: 0.6250 - val_loss: 0.6727 - val_accuracy: 0.5000
Epoch 2/10
6/6 [==============================] - 46s 8s/step - loss: 0.5205 - accuracy: 0.7865 - val_loss: 0.4454 - val_accuracy: 0.8667
Epoch 3/10
6/6 [==============================] - 46s 8s/step - loss: 0.4842 - accuracy: 0.7812 - val_loss: 0.5215 - val_accuracy: 0.8333
Epoch 4/10
6/6 [==============================] - 46s 8s/step - loss: 0.4230 - accuracy: 0.7917 - val_loss: 0.2852 - val_accuracy: 0.9500
Epoch 5/10
6/6 [==============================] - 48s 8s/step - loss: 0.2249 - accuracy: 0.9115 - val_loss: 0.1837 - val_accuracy: 0.9667
Epoch 6/10
6/6 [==============================] - 49s 8s/step - loss: 0.1679 - accuracy: 0.9375 - val_loss: 0.2438 - val_accuracy: 0.9667
Epoch 7/10
6/6 [==============================] - 47s 8s/step - loss: 0.1735 - accuracy: 0.9583 - val_loss: 0.0847 - val_accuracy: 0.9667
Epoch 8/10
6/6 [==================

In [ ]:
model.save('model_adv.h5')

In [11]:
model.evaluate(train_generator)

7/7 [==============================] - 14s 2s/step - loss: 0.1252 - accuracy: 0.9598


[0.1251789778470993, 0.9598214030265808]

In [12]:
model.evaluate(validation_generator)

2/2 [==============================] - 4s 2s/step - loss: 0.0794 - accuracy: 0.9833


[0.0793876126408577, 0.9833333492279053]

### Test Images

In [13]:
model = load_model('model_adv.h5')

In [14]:
model.predict?

In [15]:
import os

In [16]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [17]:
y_actual = []
y_test = []

In [21]:
for i in os.listdir("./Val/Normal/"):
    img = image.load_img("./Val/Normal/"+i, target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    p = (model.predict(img) > 0.5).astype("int32")
    print(p[0][0])
    y_test.append(p[0,0])
    y_actual.append(1)

1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0


In [22]:
for i in os.listdir("./Val/Covid/"):
    img = image.load_img("./Val/Covid/"+i, target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    p = (model.predict(img) > 0.5).astype("int32")
    print(p[0][0])
    y_test.append(p[0,0])
    y_actual.append(0)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [ ]:
y_actual = np.array(y_actual)

In [ ]:
y_test = np.array(y_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(y_actual, y_test)

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(cm, cmap='plasma', annot=True)